In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys
from glob import glob
sys.path.append('..')
from configs import *
import configs
from tqdm import tqdm
from utils import *
import importlib

In [3]:
importlib.reload(configs)

<module 'configs' from '../configs.py'>

In [4]:
dpath = '%s/%s/*_0.txt' % (DPATH, ROUTES)
print(dpath)
rfiles = glob(dpath)
print(len(rfiles))

/home/ubuntu/traffic/data/stopcodes_sequence/*_0.txt
328


In [5]:
all_routes = []
for rf in rfiles:
    with open(rf) as fl:
        stops = fl.read().split('\n')[:-1]
    if len(stops) > 5:
        all_routes.append(dict(name=fileName(rf), stops=stops))

In [6]:
slens = [len(st['stops']) for st in all_routes]
print(max(slens), np.mean(slens), min(slens))
print(len(all_routes))

114 39.61180124223603 7
322


In [7]:
dpath = '%s/%s/**/*.csv' % (DPATH, SPEEDS)
print(dpath)
segfiles = glob(dpath)
print(len(segfiles))
scheck = {}
for sf in segfiles:
    scheck[fileName(sf)] = True

/home/ubuntu/traffic/data/avgspeeds-full-ts-xclude/**/*.csv
16728


In [8]:
all_segs = []
for route in all_routes:
    stops = route['stops']
    sl = []
    for si in range(len(stops) - 1):
        sl.append(segkey(stops[si], stops[si+1]))
    route['segs'] = sl
    all_segs.append(route)

In [9]:
safe_segs = []
for route in all_segs:
    segs = route['segs']
    if all([entry in scheck for entry in segs]):
        safe_segs.append(route)
print(len(safe_segs))

245


In [10]:
print(safe_segs[0]['name'], safe_segs[0]['segs'])

M104_0 ['405374-404919', '404919-404872', '404872-404846', '404846-401239', '401239-405393', '405393-401016', '401016-400747', '400747-403271', '403271-400750', '400750-403103', '403103-403104', '403104-403106', '403106-403107', '403107-403108', '403108-403109', '403109-403110', '403110-403111', '403111-403112', '403112-405248', '405248-403114', '403114-403115', '403115-403116', '403116-405394', '405394-403118', '403118-403119', '403119-404260', '404260-403122', '403122-404262', '404262-403125', '403125-403126', '403126-403127', '403127-401445', '401445-903089']


In [53]:
for route in safe_segs:
    with open('data/routes/%s.txt' % route['name'], 'w') as fl:
        fl.write('name,seg\n')
        for seg in route['segs']:
            fl.write('%s,%s\n' % (route['name'], seg))

In [64]:
for route in tqdm(safe_segs):
    vmat = -np.ones((13248, len(route['segs'])))
    for si, seg in enumerate(route['segs']):
        with open('%s/%s/%s/%s.csv' % (DPATH, SPEEDS, seg[0], seg)) as fl:
            _ = fl.readline() # header
            line = fl.readline()
            li = 0
            while line:
                line = line.strip('\n').split(',')
                vel = None if line[1] == '' else float(line[1])
                vmat[li, si] = vel
                line = fl.readline()
                li += 1
    savef = 'data/history/%s.npy' % route['name']
    np.save(savef, vmat)

100%|██████████| 245/245 [16:24<00:00,  4.02s/it]
